In [ ]:
import numpy as np
import pandas as pd
import nltk
import re
import time
import openai
openai.api_key = "TYPE HERE YOUR API KEY"

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## Determination of seniority level and relevant experience using NLTK

In [ ]:
# load data and make new columns
df = pd.read_csv('LI_PM_jobDesc.csv', delimiter = ',')
df['seniorityLevel'] = ''
df['relExperience'] = ''

# definition of keywords 
seniority_words = ['junior', 'entry', 'intership', 'medior', 'senior', 'lead', 'director', 
                   'pályakezdő', 'szenior', 'osztályvezető']

# tokenization the jobDescriptions
for index, row in df.iterrows():
    tokens = nltk.word_tokenize(row['jobDescription'].lower())

    # definition of seniority levels
    for token in tokens:
        if token in seniority_words:
            df.loc[index, 'seniorityLevel'] = token
            break

    # definition of relevant experience
    matches = re.findall(r'(\d+\+*-*\d*)\s*(?:years|yr|év)', row['jobDescription'].lower())
    if matches:
        df.loc[index, 'relExperience'] = ', '.join(matches)

# print results
df[['companyName', 'seniorityLevel', 'relExperience']]

In [ ]:
# fix incorrect data in relExperience column

df.at[18, 'relExperience'] = '2+'
df.at[24, 'relExperience'] = '5-10'
df.at[67, 'relExperience'] = '1-2'
df.at[80, 'relExperience'] = ''
df.at[104, 'relExperience'] = '2+'
df.at[105, 'relExperience'] = ''
df.at[117, 'relExperience'] = ''
df.at[120, 'relExperience'] = '3'
df.at[134, 'relExperience'] = '2'

df[['companyName', 'seniorityLevel', 'relExperience']]


## Determination of seniority level and relevant experience using AI

In [ ]:
#Create a new dataframe
LI_PM_jobad = pd.read_csv('LI_PM_jobDesc.csv', delimiter = ',')
df2 = pd.DataFrame()
df2['companyName'] = LI_PM_jobad['companyName']
df2['jobDescription'] = LI_PM_jobad['jobDescription']
df2['seniorityLevel'] = ''

#definition of seniority_words
seniority_words = ['junior', 'entry', 'intership', 'medior', 'senior', 'lead', 'director', 
                   'pályakezdő', 'szenior', 'osztályvezető']

#iterate through the dataframe
for index, row in df2.iterrows():
    
    prompt_seniorityLevel = row['jobDescription'] + "\n --- \n This is a product manager job description.\
                 Decribe the seniority level of this job. Choose one from these levels \
                 (and write just this one world into the dataframe):" + ','.join(seniority_words) + "Answer the question as truthfully as possible using the provided context. \
                 If the seniority level cannot be clearly determined based on the description, use the value 'Unknown'."
        
    response_seniorityLevel = openai.ChatCompletion.create(
        model = "gpt-4",
        messages = [
            {"role": "system", "content": "You are a helpful data science assistant."},
            {"role": "user", "content": prompt_seniorityLevel}],
        max_tokens=50,
        temperature=0)
    
    message_seniorityLevel = response_seniorityLevel['choices'][0]['message']['content'].strip()

    df2.loc[index, 'seniorityLevel'] = message_seniorityLevel
    
    time.sleep(0.5)
        
    if index % 10 == 0:
        print(f"Processed row {index} for company {row['companyName']} with seniority level: {message_seniorityLevel}")
        
#print results
df2[['companyName', 'seniorityLevel']]